In [1]:
# Module imports
import os
from bs4 import BeautifulSoup
import pickle
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
# Let's try setting up a class Case with properties "corpus", "wordcount", "casenum""
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

class Case:
    def __init__(self, textfile):
        assert os.path.isfile(textfile) == True
        # We start out with just the raw text string from the pdf
        # We should also initialize the casenumber attribute?
        with open(textfile, "rb") as file: 
            self.text = pickle.load(file)
            self.casenumber = os.path.splitext(os.path.split(textpath)[1])[0]
    # We should have a method for doing text cleanup and a method for 
    # count vectorization
    def make_corpus(self):
        '''Adds corpus attribute to the Case object by cleaning
        the raw text string'''
        # Make text lowercase: 
        text = self.text
        text = text.lower()
        # Strip all newlines: 
        text = re.sub('\n', '', text)
        self.corpus = text
    def make_unigram_matrix(self):
        '''Adds frequency-matrix-of-unigrams attribute to Case object. Requires
        that corpus attribute already exists.'''
        # currently redundant with ngram matrix
        cv = CountVectorizer(input='content',stop_words = 'english')
        freq_cv = cv.fit_transform([self.corpus])
        freq_matrix = pd.DataFrame(freq_cv.toarray(), columns = cv.get_feature_names())
        # Sort the words by frequency, most frequent first
        # freq_matrix = freq_matrix.sort_values(by=0,axis=1, ascending=False)
        # freq_matrix = freq_matrix.transpose()
        self.unigram_matrix = freq_matrix
    def make_ngram_matrix(self, n):
        '''Adds frequency-matrix-of-ngrams attribute to Case object. Requires
        that corpus attribute already exists.'''
        cv = CountVectorizer(input='content',stop_words = 'english',strip_accents='unicode',ngram_range=(1,n))
        freq_cv = cv.fit_transform([self.corpus])
        freq_matrix = pd.DataFrame(freq_cv.toarray(), columns = cv.get_feature_names())
        # Sort the words by frequency, most frequent first
        # freq_matrix = freq_matrix.sort_values(by=0,axis=1, ascending=False)
        # freq_matrix = freq_matrix.transpose()
        # note: transposing matrix makes the to_string method produce
        # a nice output, but makes indexing by ngram slightly harder?
        self.ngram_matrix = freq_matrix
    
    def make_refined_corpus(self):
        regex_spaces = '(\s+)'
        self.refined_corpus = re.sub(r'(\s+)', ' ', self.corpus)
            
    def save_to_file(self,filepath):
        with open(filepath, 'w') as f:
            f.write(self.refined_corpus)
  
        

One thing that's caused problems using the corpus is that the whitespace is very irregular, meaning matching expressions was impeded by the presence of tabs, returns, and long strings of space between words.

In [28]:
with open('short.bin', 'rb') as file: 
    shortlist = pickle.load(file)

In [35]:
def make_refined_corpus(string):
    regex_spaces = '(\s+)'
    refined_corpus = re.sub(r'(\s+)', ' ', string)
    refined_corpus = re.sub(r'[^\x00-\x7F]+', '', refined_corpus)
    return refined_corpus



In [27]:
test_case = shortlist[0]
test_case.corpus


'filed 4/2/12  p. v. khrayan ca2/2 not to be published in the official reports  california rules of court, rule 8.1115(a), prohibits courts and parties from citing or relying on opinions not certified for publication or ordered published, except as specified by rule 8.1115(b).  this opinion has not been certified for publication or ordered published for purposes of rule 8.1115.  in the court of appeal of the state of california  second appellate district  division two   the people,   plaintiff and respondent,   v.  arutyun khrayan,   defendant and appellant.        b213582        (los angeles county       super. ct. nos. ba255474, ba255302)      appeal from a judgment of the superior court of los angeles county.  kathleen kennedy, judge.  affirmed.   geragos & geragos and mark j. geragos for defendant and appellant.   kamala d. harris, attorney general, dane r. gillette, chief assistant attorney general, lance e. winters, assistant attorney general, linda c. johnson, robert david breto

In [41]:
clean_corpus_list = []
folder = os.path.join(os.path.abspath(''),'refined_corpuses')
for case in shortlist:
    clean_corpus = make_refined_corpus(case.corpus)
    writefile = os.path.join(folder,'%s.txt' % case.casenumber)
    with open(writefile, 'w') as f: 
        f.write(clean_corpus)
    clean_corpus_list.append(clean_corpus)

In [42]:
len(clean_corpus_list)

8474

In [46]:
casenumbers_la_felonies = []
for case in shortlist:
    casenumbers_la_felonies.append(case.casenumber)
with open('list_of_casenumbers.txt', 'wb') as f: 
    pickle.dump(casenumbers_la_felonies, f)

In [40]:
clean_corpus_list[6]

'filed 4/11/13 p. v. lozano ca2/3 not to be published in the official reports california rules of court, rule 8.1115(a), prohibits courts and parties from citing or relying on opinions not certified for publication or ordered published, except as specified by rule 8.1115(b). this opinion has not been certified for publication or ordered published for purposes of rule 8.1115. in the court of appeal of the state of california second appellate district division three the people, plaintiff and respondent, v. sammy lozano, defendant and appellant. b233393 (los angeles county super. ct. nos. ta077104 & ta080053) appeal from a judgment of the superior court of los angeles county, allen joseph webster, judge. affirmed with directions. george l. schraer for defendant and appellant. kamala d. harris, attorney general, dane r. gillette, chief assistant attorney general, lance e. winters, assistant attorney general, paul m. roadarmel, jr., stephanie a. miyoshi and william n. frank, deputy attorney

In [54]:
test_corpus_clean = clean_corpus_list[10]

In [69]:
# Get filing date
# Search for the string "filed " followed by a date
# return the date
def get_filing_date(corpus):
    strings = re.search(r'filed \d+/\d+/\d+', corpus)
    if len(strings) == 1:
        string = strings[0]
        date = string.replace('filed ','')
        l = date.split('/')
        date_info = {'date':date, 'month':int(l[0]), 'day':int(l[1]), 'year':int(l[2]) + 2000}
        return date_info
    elif len(strings) == 0: 
        print('No dates found.')
    else: 
        print('Multiple dates found.')
    return strings

In [73]:
for corps in clean_corpus_list[0:20]:
    print(get_filing_date(corps)['year'])


2012
2012
2012
2012
2012
2012
2013
2012
2012
2013
2012
2012
2012
2012
2012
2012
2012
2013
2012
2013


In [150]:
# Get defendant name
# Search for "respondent, v. " or similar followed by
# "name, defendant"
# return name
def get_defendant(corpus):
    defendant_name = re.findall(r'respondent, vs?\. .{3,100}?, defendants?\b', corpus)
    if len(defendant_name) == 0:
        defendant_name = ['oop', 'oh no']
    return defendant_name

In [151]:
defendant_search_term = r'respondent, vs?\. .+, defendants?'
p = get_defendant(test_corpus_clean[0])
p

['oop', 'oh no']

In [152]:
for corpus in clean_corpus_list:
    try:
        print(get_defendant(corpus)[0])
    except IndexError:
        print(len(get_defendant(corpus)))
        continue

respondent, v. arutyun khrayan, defendant
respondent, v. arsellers c. scott, defendant
respondent, v. terrell frazier et al., defendants
respondent, v. corey mcclelland et al., defendants
respondent, v. peter thomas rodriguez, defendant
respondent, v. nicole lenea parson, defendant
respondent, v. sammy lozano, defendant
oop
respondent, v. william edson kingsland, defendant
respondent, v. paki john bronson, defendant
respondent, v. margarito villasenor, defendant
respondent, v. johnny d. horton, defendant
respondent, v. tony thomas, defendant
respondent, v. rudy ruiz, defendant
respondent, v. david wesley cassis, defendant
respondent, v. michael barrios, defendant
respondent, v. oliver wendell james, defendant
respondent, v. arvind sinha, defendant
respondent, v. scott bradley millwee, defendant
respondent, v. jesus duran aguayo, et al., defendants
respondent, v. kehende m. lang, defendant
respondent, v. patrick edward panzarello, defendant
respondent, v. robert andrew repka, defendant


respondent, v. roberto avila, defendant
respondent, v. chris p. velasquez, defendant
respondent, v. richard kyle johnson, defendant
respondent, v. roland windsor vincent, defendant
respondent, v. michael carl helm, defendant
respondent, v. richard lavel hill, defendant
respondent, v. prince ayree, defendant
respondent, v. miguel g. hernandez et al., defendants
respondent, v. gabriel orozco, defendant
respondent, v. arthur lee smart, defendant
respondent, v. christopher stratis et al., defendants
respondent, v. kristopher thomas, defendant
respondent, v. adan mejia, defendant
respondent, v. ricky white, defendant
respondent, v. guillermo miranda, defendant
respondent, v. domingo manriques, defendant
respondent, v. joadanus jerome olivas, defendant
respondent, v. israel soto, defendant
respondent, v. david wayne spivey, defendant
respondent, v. jose l. molina, defendant
respondent, v. lamar david williams, defendant
respondent, v. bartolome nava, defendant
respondent, v. devonte ross, de

respondent, v. deante duckett, defendant
respondent, v. jamal rehman, defendant
respondent, v. ronald batiste, defendant
respondent, v. paul anthony wiley, defendant
respondent, v. valentin polyvko, defendant
respondent, v. husie outing, defendant
respondent, v. daryl lynch, defendant
respondent, v. angel juarez, defendant
respondent, v. keith maxwell, defendant
respondent, v. steve chavez, defendant
respondent, v. charles allen jackson, defendant
respondent, v. lorenzo micquell latimer, defendant
respondent, v. oscar vargas, defendant
respondent, v. william wall et al., defendants
respondent, v. rodney cyril carr, defendant
respondent, v. james landrum, sr., defendant
respondent, v. luis villalvazo et al., defendants
respondent, v. santos mejia, defendant
respondent, v. necole richards, defendant
respondent, v. tommy glenn crane, defendant
respondent, v. daniel enrique galvez, defendant
respondent, v. michael humberto munguia, defendant
respondent, v. ryan amirant, defendant
responden

respondent, v. juan francisco polasek, defendant
respondent, v. nanine nanez, defendant
respondent, v. jason st. pierre, defendant
respondent, v. martin oliveras, defendant
respondent, v. henry anthony ayala, defendant
respondent, v. myford david jenkins, defendant
respondent, v. martin sotelo, defendant
respondent, v. leon sims, defendant
respondent, v. gregory vernell farmer, defendant
respondent, v. gary wright, defendant
respondent, v. larry arthur list, defendant
respondent, v. randy doby et al., defendants
respondent, v. norma lilian cortez et al., defendants
respondent, v. enrique silva santoyo, defendant
respondent, v. michelle traniece davis, defendant
respondent, v. juan carlos huezo, defendant
respondent, v. william ruiz, defendant
respondent, v. salvador merced, defendant
respondent, v. quintin maddox, jr., defendant
respondent, v. stacey marie barker, defendant
respondent, v. levitius daniel wright, defendant
respondent, v. herbert henderson, defendant
respondent, v. jason

respondent, v. daniel belmonte, defendant
respondent, v. jaron deandre lucien, defendant
respondent, v. jimmy encinas, defendant
respondent, v. luis enrique fernandez, defendant
respondent, v. edgar a. carrillo, defendant
respondent, v. joseph sam leon, defendant
oop
respondent, v. davione mcdowell, defendant
respondent, v. rudolph edwards, defendant
respondent, v. steve chavez, defendant
respondent, v. adrian rodriguez, defendant
respondent, v. carlos a. rodriguez, defendant
respondent, v. wesley stanley cotton, defendant
respondent, v. artist dwayne hardy, iii, defendant
respondent, v. tanya christine byer, defendant
respondent, v. charles williams, defendant
respondent, v. herber morales, defendant
respondent, v. luis miguel noriega, defendant
respondent, v. valentino rodriguez, defendant
respondent, v. alberto h. fonseca, defendant
respondent, v. joshua pablo rosales, defendant
respondent, v. donyell ladale butler, defendant
oop
respondent, v. christopher laurence barnum, defendant

...

respondent, v. carlos daniels, defendant
respondent, v. george parker, defendant
respondent, v. joaquin linares, defendant
respondent, v. cornell aaron brown, defendant
respondent, v. r.j., defendant
respondent, v. margie valeriano dikit, defendant
respondent, v. alvin ray shaw, jr., defendant
respondent, v. joe peter montoya, defendant
respondent, v. misael agid martinez, defendant
respondent, v. john p. firestone-kelly, defendant
respondent, v. harry lee ollie, defendant
respondent, v. wesley garcia, defendant
respondent, v. jose pepe mitchell, defendant
respondent, v. cuauhtemoc martinez, defendant
respondent, v. victor weathers, defendant
respondent, v. peter l. burchett, defendant
respondent, v. daveon travon toutar, defendant
respondent, v. hall lycurgus johnson, defendant
respondent, v. artis wilson, defendant
respondent, v. edgar arellano, defendant
respondent, v. christopher propps, defendant
respondent, v. agustin perez, defendant
respondent, v. jemar welch, defendant
respond

respondent, v. frankie calanche lopez, defendant
respondent, v. pierre lakell morris, defendant
respondent, v. brittany ann ingrassi, defendant
respondent, v. lloyd carr, defendant
respondent, v. dejuan lamont robertson, defendant
respondent, v. leslie croaker, defendant
respondent, v. nathaniel cyprian, defendant
respondent, v. jae jeong lyu, defendant
respondent, v. willie fred stephens, sr., defendant
respondent, v. angelo camacho, defendant
respondent, v. chaka grossman, defendant
respondent, v. arthur manuel villegas, defendant
respondent, v. matthew louis johnson, defendant
oop
respondent, v. anthony lamar mitchell, defendant
respondent, v. cody william leffler, defendant
respondent, v. anthony jonathan nila, defendant
respondent, v. omar delgado, defendant
respondent, v. ricardo avila, defendant
respondent, v. gabriel joseph casados, defendant
respondent, v. jack g. dadanian, defendant
respondent, v. sandra ritchie, defendant
respondent, v. robert mcarthur barnes, jr., defendant

respondent, v. leonardo salmeron, defendant
respondent, v. norman elder grundy, defendant
respondent, v. joshua hurtado, defendant
respondent, v. manuel jesus serrano, defendant
respondent, v. arturo acevedo, defendant
respondent, v. ernest j. casique, defendant
respondent, v. raul martinez, defendant
respondent, v. david worley, defendant
respondent, v. ronnie lee roman, defendant
respondent, v. adekunle olobayo-aisony, defendant
respondent, v. ezekiel simon, defendant
respondent, v. scott brian garcia, defendant
respondent, v. shvonte abdual bogard, defendant
respondent, v. jesse delgado, defendant
respondent, v. aaron arredondo, defendant
respondent, v. jordan koziol et al., defendants
respondent, v. alon oneil foster, defendant
respondent, v. hector bravo, defendant
respondent, v. gerald w. woods, defendant
respondent, v. michael clayton perkins, defendant
respondent, v. rhodanker mcguffie, defendant
respondent, v. anucha suwannangkul, defendant
respondent, v. michael parks, defend

respondent, v. franky robert mendez, defendant
respondent, v. john burns, defendant
respondent, v. christopher harrison goodine, defendant
respondent, v. jerome major carter, defendant
respondent, v. david gordon mountford, defendant
respondent, v. clyde richards, defendant
respondent, v. carlos cardoso, defendant
respondent, v. deante reginald smith, defendant
respondent, v. albert florencio chagollan, defendant
respondent, v. diane christine vigil, defendant
respondent, v. ramon armijo, defendant
respondent, v. jerome dean, defendant
respondent, v. trayvion rene thomas, defendant
respondent, v. tavares londell mcintosh, defendant
respondent, v. jason jimenez, defendant
respondent, v. chester alan duncan, defendant
respondent, v. romelio corelio espinoza, defendant
respondent, v. george alfaro, defendant
respondent, v. edwin phillips, defendant
respondent, v. tony smith et al, defendants
respondent, v. robert veiga, defendant
respondent, v. eduardo samayoa salguero, defendant
responde

respondent, v. phillip gallegos, defendant
respondent, v. juan carlos aquino, defendant
respondent, v. manuel anthony garibay, defendant
respondent, v. theodore justin bowers, defendant
respondent, v. lester mcdoughtery, defendant
respondent, v. demetris cornelious, defendant
respondent, v. michael anthony blankenship, defendant
respondent, v. troy m. hill, defendant
respondent, v. mark anthony gallegos, defendant
respondent, v. gregory anderson, defendant
respondent, v. rafiki smith, defendant
respondent, v. serafin torres urbina, defendant
respondent, v. robert james spells, defendant
respondent, v. robert clay woods, defendant
respondent, v. garveia brandon freeny, et al., defendants
respondent, v. bulmario g. torres, defendant
respondent, v. salvador hernandez, defendant
respondent, v. andrea almanza alfaro et al., defendants
respondent, v. james kelly norton, defendant
respondent, v. gerry mclean bailey, defendant
respondent, v. ismael alejandro sepulveda, defendant
respondent, v.

In [ ]:
# Get defendant attorney
# search for ". names " in 

In [59]:
s = get_filing_date(test_corpus_clean[0:2000])
s

[]

In [62]:
re.findall(r'filed \d+/\d+/\d+', test_corpus_clean)

['filed 8/22/12']

In [61]:
test_corpus_clean[0:2000]

' filed 8/22/12 p. v. villasenor ca2/7 not to be published in the official reports california rules of court, rule 8.1115(a), prohibits courts and parties from citing or relying on opinions not certified for publication or ordered published, except as specified by rule 8.1115(b). this opinion has not been certified for publication or ordered published for purposes of rule 8.1115. in the court of appeal of the state of california second appellate district division seven the people, plaintiff and respondent, v. margarito villasenor, defendant and appellant. b234343 (los angeles county super. ct. nos. ka093275, ka090956) appeal from a judgment and an order of the superior court of los angeles county, mike camacho, judge. judgment affirmed; order affirmed as modified. david l. polsky, under appointment by the court of appeal, for defendant and appellant. kamala d. harris, attorney general, dane r. gillette, chief assistant attorney general, lance e. winters, assistant attorney general, jam

In [163]:
attorney = r'(\. .{3,100}? for defendants?)'
appointed = r'(under appointment)'
outcome = r'(\. judgm?en?t? .{3,100}?\.)'


In [164]:
def search_corpus(corpus, string): 
    found = re.findall(string, corpus[0:2000])
    if len(found) == 0:
        print('Not found.')
    return found
     

In [165]:
for corpus in clean_corpus_list[0:20]:
    print((search_corpus(corpus, attorney), search_corpus(corpus, appointed), search_corpus(corpus, outcome)))

Not found.
Not found.
(['. kathleen kennedy, judge. affirmed. geragos & geragos and mark j. geragos for defendant'], [], [])
Not found.
(['. affirmed. richard c. neuhoff and barbara a. zuras, under appointment by the court of appeal, for defendant'], ['under appointment'], [])
Not found.
(['. affirmed as modified. joseph shipp, under appointment by the court of appeal, for defendant', '. edward j. haggerty, under appointment by the court of appeal, for defendant'], ['under appointment', 'under appointment'], [])
Not found.
(['. affirmed as modified. sarah a. stockwell, under appointment by the court of appeal, for defendant', '. thomas owen, under appointment by the court of appeal, for defendant'], ['under appointment', 'under appointment'], [])
Not found.
(['. appeal dismissed. gary v. crooks, under appointment by the court of appeal, for defendant'], ['under appointment'], [])
Not found.
(['. charles a. chung, judge. affirmed. karyn h. bucur, under appointment by the court of appeal